In [27]:
import numpy as np
from torch.utils.data import DataLoader
import torch
from flytracker.tracker import _run, _initialize, _localize
from flytracker.analysis import annotate
from flytracker.io import VideoDataset
from flytracker.preprocessing.preprocessing import preprocessing_noremap, preprocessing_torch
from flytracker.localization.blob import localize_blob, default_blob_detector_params
from flytracker.localization.kmeans import localize_kmeans_torch, localize_kmeans
from flytracker.tracking import tracking
from flytracker.analysis import post_process


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
movie_path = "../../data/experiments/bruno/videos/seq_1.mp4"


mask = torch.ones((1080, 1280), dtype=bool)
mask[:160, :] = 0
mask[-170:, :] = 0
mask[:, :300] = 0
mask[:, -230:] = 0

mask[:220, :400] = 0
mask[:230, -300:] = 0
mask[-250:, :370] = 0
mask[830:, 970:] = 0

In [40]:
dataset = VideoDataset(movie_path)
loader = DataLoader(dataset, batch_size=None, pin_memory=True)

In [41]:
preprocessor_ini = preprocessing_torch(mask, torch.tensor(255, dtype=torch.uint8))
initial_position, initial_frame = _initialize(loader, localize_blob, (default_blob_detector_params(), preprocessor_ini), 100)

In [42]:
preprocessor_main= preprocessing_torch(mask.cuda(), torch.tensor(255, dtype=torch.uint8).cuda())

In [43]:
%%time
locations = _localize(loader, localize_kmeans_torch, (preprocessor_main, 120, 'cuda'), initial_position, 10000)

Done with frame 0
Done with frame 1000
Done with frame 2000
Done with frame 3000
Done with frame 4000
Done with frame 5000
Done with frame 6000
Done with frame 7000
Done with frame 8000
Done with frame 9000
Done with frame 10000
CPU times: user 10min 18s, sys: 10.8 s, total: 10min 28s
Wall time: 37.3 s


In [44]:
1e4 / 40

250.0

10002